In [1]:
import pandas as pd 
import numpy as np 
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

SEQ_LEN = 10
BATCH_SIZE = 32
EPOCHS = 10
LR = 0.001
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
df = pd.read_csv('/Users/g/Desktop/Projects/Machine-Learning/outputs/processed_data/BTC_USD_processed.csv')
print(df.head())


        Price        Close         High          Low         Open  \
0  2018-02-03  9174.910156  9430.750000  8251.629883  8852.120117   
1  2018-02-04  8277.009766  9334.870117  8031.220215  9175.700195   
2  2018-02-05  6955.270020  8364.839844  6756.680176  8270.540039   
3  2018-02-06  7754.000000  7850.700195  6048.259766  7051.750000   
4  2018-02-07  7621.299805  8509.110352  7236.790039  7755.490234   

         Volume        RSI  MACD_12_26_9  MACDh_12_26_9  MACDs_12_26_9  \
0  7.263790e+09  32.239444  -1510.351215     -57.720049   -1452.631166   
1  7.073550e+09  28.425279  -1568.807746     -92.941264   -1475.866482   
2  9.285290e+09  23.936093  -1702.166800    -181.040254   -1521.126545   
3  1.399980e+10  31.025208  -1723.536125    -161.927664   -1561.608461   
4  9.169280e+09  30.516343  -1731.222807    -135.691476   -1595.531330   

     BBL_5_2.0    BBM_5_2.0     BBU_5_2.0  BBB_5_2.0  BBP_5_2.0        SMA_20  \
0  8387.273257  9500.719922  10614.166586  23.439206   0.35

In [5]:
features = [
    "RSI", "MACD_12_26_9", "MACDs_12_26_9",
    "BBL_5_2.0", "BBM_5_2.0", "BBU_5_2.0",
    "SMA_20", "EMA_20", "OBV"
]
target = 'Target_1d'

class TimeSeriesDataset(Dataset):
    def __init__(self, df):
        self.X = []
        self.y = []
        for i in range(len(df) - SEQ_LEN):
            seq_x = df[features].iloc[i:i+SEQ_LEN].values
            seq_y = df[target].iloc[i+SEQ_LEN]
            self.X.append(seq_x)
            self.y.append(seq_y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)
        

In [6]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size =64, num_layers=2):
        super().__init__()
        self.lstm= nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 2)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1])
        return out

model = LSTMModel(input_size=len(features)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [12]:
df = pd.read_csv('/Users/g/Desktop/Projects/Machine-Learning/outputs/labeled_data/BTC_USD_processed.csv')
df.dropna(subset=features + [target], inplace=True)
df = df.reset_index(drop=True)
train_size = int(0.8 * len(df))
train_df = df[:train_size]
val_df = df[train_size:]
train_loader = DataLoader(TimeSeriesDataset(train_df), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TimeSeriesDataset(val_df), batch_size=BATCH_SIZE)

In [13]:
for epoch in range(EPOCHS):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
        optimizer.zero_grad()
        outputs= model(X_batch)
        loss=criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {loss.item():.4f}")

Epoch 1/10 | Loss: 0.6889
Epoch 2/10 | Loss: 0.6914
Epoch 3/10 | Loss: 0.6875
Epoch 4/10 | Loss: 0.6939
Epoch 5/10 | Loss: 0.6914
Epoch 6/10 | Loss: 0.6979
Epoch 7/10 | Loss: 0.6961
Epoch 8/10 | Loss: 0.6948
Epoch 9/10 | Loss: 0.6943
Epoch 10/10 | Loss: 0.6796


In [14]:
model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch = X_batch.to(DEVICE)
        preds = model(X_batch).argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_targets.extend(y_batch.numpy())

acc = np.mean(np.array(all_preds) == np.array(all_targets))
print(f"\n LSTM Accuracy (BTC/USD, 1-Day): {acc:.4f}")


 LSTM Accuracy (BTC/USD, 1-Day): 0.5212
